In [1]:
import pandas as pd
import json

In [2]:
start_urls = []
with open('../paruvendu_api/outputs/data_harvest_full.json','r') as sample:
    for line in sample:
        start_urls.append(json.loads(line.strip()))

In [3]:
f = open('../paruvendu_api/outputs/brands.json')
brands = json.load(f)

In [4]:
data = pd.DataFrame.from_dict(start_urls)

In [5]:
df = data.copy()

In [6]:
df.head()

,url,status,location,version,price,year,mileage,fuel_type,emission,fuel_usage,transmission,door_nb,technical_power,seat_nb,actual_power,body_colour,body_type
0,https://www.paruvendu.fr/a/voiture-occasion/da...,200,dissay (86130),version duster tce 130 fap 4x2 prestige,prix 17 600 €,année février 2019 le comparateur d'assurance ...,kilométrage 18 300 km,energie essence,N/A,N/A,transmission manuelle,nb de portes 4 portes avec hayon,puissance fiscale 7 cv,nombre de places 5 places,130,blanc,4x4
1,https://www.paruvendu.fr/a/voiture-occasion/mi...,200,59240 dunkerque,version cooper s red hot chili 192 cv bva,prix 24 990 €,année mai 2018 le comparateur d'assurance pour...,kilométrage 47 500 km,energie essence,emissions de co2 119 g/km,N/A,transmission automatique,nb de portes 2 portes avec hayon,puissance fiscale 10 cv,nombre de places 4 places,192,rouge foncé,berline
2,https://www.paruvendu.fr/a/voiture-occasion/ni...,200,53000 laval,version 1.5 dci 110ch fap ultimate edition,prix 8 490 €,année octobre 2013 le comparateur d'assurance ...,kilométrage 166 400 km,energie diesel,emissions de co2 129 g/km,N/A,transmission manuelle,nb de portes 4 portes avec hayon,puissance fiscale 6 cv,nombre de places 5 places,110,gris,break
3,https://www.paruvendu.fr/a/voiture-occasion/ni...,200,belley (01300),version qashqai 1.6 dci 130 xtronic tekna,prix 16 900 €,année octobre 2016 le comparateur d'assurance ...,kilométrage 61 500 km,energie diesel,emissions de co2 128 g/km,consommation mixte 4.9 litres / 100 km,transmission automatique,nb de portes 4 portes avec hayon,puissance fiscale 7 cv,nombre de places 5 places,130,gris foncé,4x4
4,https://www.paruvendu.fr/a/voiture-occasion/mi...,200,30200 bagnols-sur-cèze,version one d 95 ch edition blackfriars,prix 17 900 €,année février 2018 le comparateur d'assurance ...,kilométrage 58 529 km,energie diesel,emissions de co2 99 g/km,N/A,transmission manuelle,nb de portes 4 portes avec hayon,puissance fiscale 5 cv,nombre de places 5 places,95,gris,berline


## Pré-nettoyage

In [7]:
import re 
import numpy as np

In [8]:
def extract_data(column, index):
    url_list = df[column].tolist()
    
    brand_model_list = []
    for url in url_list:
        if 'N/A' not in url:
            url = url.split(' ')[index]
            brand_model_list.append(url)
        else:
            brand_model_list.append(url)
            
    return brand_model_list

In [9]:
columns = ['fuel_usage', 'technical_power']
for col in columns:
    df[col] = extract_data(col, 2)

In [10]:
df['year'] = df['year'].apply(lambda x: re.sub("[^\d\.]", "", f"{x}")).replace({'':np.nan})
df['year'] = df['year'].astype('string').str[:4].astype('Int64')

In [11]:
columns = ['emission', 'door_nb', 'seat_nb']
for col in columns:
    df[col] = extract_data(col, 3)

In [12]:
url_list = df['url'].tolist()
brand_model_list = []
for url in url_list:
    url = url.split('/')
    brand_model = url[5] + "/" + url[6]
    brand_model_list.append(brand_model)
df['version'] = brand_model_list

In [13]:
columns = ['location', 'price', 'mileage']

for col in columns:  
    df[col] = df[col].apply(lambda x: re.sub("[^\d\.]", "", f"{x}"))
    df[col] = df[col].astype('Int64', errors='ignore')

In [14]:
df = df.replace({"N/A": np.nan, '': np.nan})
df['fuel_type'] = df['fuel_type'].str.replace('energie ', '')
df['transmission'] = df['transmission'].str.replace('transmission ', '')
del df['url']
del df['status']
del df['fuel_usage']

In [15]:
df.head()

,location,version,price,year,mileage,fuel_type,emission,transmission,door_nb,technical_power,seat_nb,actual_power,body_colour,body_type
0,86130,dacia/duster,17600,2019,18300,essence,NaN,manuelle,4,7,5,130,blanc,4x4
1,59240,mini/cooper,24990,2018,47500,essence,119,automatique,2,10,4,192,rouge foncé,berline
2,53000,nissan/qashqai,8490,2013,166400,diesel,129,manuelle,4,6,5,110,gris,break
3,01300,nissan/qashqai,16900,2016,61500,diesel,128,automatique,4,7,5,130,gris foncé,4x4
4,30200,mini/cooper,17900,2018,58529,diesel,99,manuelle,4,5,5,95,gris,berline


### Enregistrement du dataframe

##### Élimination des outliers

In [16]:
del df['body_colour']
df = df[df['fuel_type'] != "hybride"]
df = df[df['fuel_type'] != "autres"]
df = df[df['fuel_type'] != "electrique"]
df = df[df['fuel_type'] != "gpl ou gnl"]
df = df[df['transmission'] != "semi automatique"]
df = df[df['transmission'] != "autres"]
df = df[df['door_nb'] != "6"]
df = df[df['seat_nb'] != "54"]
df = df[df['body_type'] != "pick-up"]
df = df[df['body_type'] != "véhicule"]
df = df[df['body_type'] != "fourgon"]
df = df[df['version'].map(df['version'].value_counts()) > 1]
df = df[df['year'].map(df['year'].value_counts()) > 1]
df['location'] = df['location'].str[0:2]

In [17]:
df.to_csv('../paruvendu_api/outputs/raw_dataset.csv', index=False, encoding="utf-8")

## Phase exploratoire

In [18]:
columns = ['location', 'version', 'price', 'year', 'mileage', 'fuel_type', 'emission', 'transmission', 'door_nb', 'technical_power', 'seat_nb', 'actual_power', 'body_type']
for col in columns:
    print(df[col].value_counts())

31    1626
06     679
69     525
33     482
59     471
      ... 
20       5
09       3
48       2
58       2
15       1
Name: location, Length: 96, dtype: int64
bmw/serie-1               216
opel/corsa                209
nissan/juke               208
ford/focus                206
seat/leon                 196
                         ... 
lexus/nx                    2
ford/transit-connect        2
renault/kangoo-express      2
peugeot/508-sw              2
lexus/ct                    2
Name: version, Length: 209, dtype: int64
11990    205
19990    184
17990    169
12990    165
18990    163
        ... 
16650      1
18150      1
15150      1
43467      1
1400       1
Name: price, Length: 2406, dtype: int64
2018    2438
2019    2214
2021    1974
2017    1808
2020    1710
2016    1382
2015     933
2014     632
2013     469
2012     433
2011     290
2022     244
2010     218
2009     199
2008     140
2007     128
2006      85
2005      67
2004      45
1999      29
2002      26
2003      2

In [19]:
a

NameError: name 'a' is not defined

### Création d'une colonne modèle

In [ ]:
models = [element for key, value in brands.items() for element in value]

In [ ]:
# for element in df['version']:
#     for model in models:
#         try:
#             model = int(model)
#         except:
#             model = str(model)
#             if model in element:
#                 print(f'{element}, {model.lower()}')

## Analyse de la forme des données

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.dtypes.value_counts()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(df.isna(), cbar=False)

In [ ]:
(df.isna().sum()/df.shape[0]).sort_values(ascending=False)

## Analyse du Fond
### Examen de la colonne target

In [ ]:
df.head()

In [ ]:
df['price'].describe()

In [ ]:
df['version'].value_counts(normalize=True)

In [ ]:
# # Some data
# labels = 'Frogs', 'Hogs', 'Dogs', 'Logs'
# fracs = [15, 30, 45, 10]

# # Make figure and axes
# fig, axs = plt.subplots(1, 2)

# # A standard pie plot
# axs[0].pie(fracs, labels=labels, autopct='%1.1f%%', shadow=True)

# # Shift the second slice using explode
# axs[1].pie(fracs, labels=labels, autopct='%.0f%%', shadow=True,
#               explode=(0, 0.1, 0, 0))

# plt.show()

In [ ]:
df['version'].value_counts()

In [ ]:
df['body_colour'].value_counts()

In [ ]:
df['year'].value_counts().plot.pie()

In [ ]:
df['fuel_type'].value_counts().plot.pie()

In [ ]:
df.groupby('fuel_type')['price'].mean().plot.pie()

In [ ]:
df['transmission'].value_counts().plot.pie()

In [ ]:
df.groupby('transmission')['price'].mean().plot.pie()

### Histogrames des variables continues

In [ ]:
df_float = df[columns].astype('float64', errors='ignore').iloc[:, 1:]
for col in df_float.select_dtypes('float64'):
    plt.figure(figsize=(10,4))
    sns.histplot(df_float[col], kde=True, stat='density')

### Relation Variables / Variables

In [ ]:
sns.pairplot(df)

In [ ]:
#correlation matrix
corrmat = df.corr()
f, ax = plt.subplots(figsize=(16,10))
mask = np.triu(np.ones_like(corrmat, dtype=np.bool))
sns.heatmap(corrmat, mask=mask, vmax=.8, square=True, cbar=True, annot=True, fmt='.3f', annot_kws={'size': 10});
ax.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':14}, pad=16)

In [ ]:
#correlation matrix
corrmat = df.corr()
sns.clustermap(corrmat, fmt='.1f');
ax.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':14}, pad=16)

In [ ]:
df_scatter = df.copy()
df_scatter = df_scatter.dropna()

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
df_scatter.plot.scatter(x = 'year', y = 'price')

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
df_scatter.plot.scatter(x = 'mileage', y = 'price')